# Image Classification of ATLAS Calorimeter Topo-Clusters: Let's Try 2D CNNs

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import libraries and some constants

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import uproot as ur
import atlas_mpl_style as ampl
ampl.use_atlas_style()
print(ur.__version__)


4.0.8


In [3]:
path_prefix = '/gpfs/share/home/cdimitri/ML4Pions/LCStudies/'
plotpath = path_prefix+'classifier/Plots/'
modelpath = path_prefix+'classifier/Models/'

In [4]:
# import our resolution utilities

import sys
sys.path.append(path_prefix)

from  util import resolution_util as ru
from  util import plot_util as pu
from  util import ml_util as mu

In [5]:
inputpath = '/cephfs/user/cdimitri/ML4Pions/'
rootfiles = ["pi0_subset", "pi_charged_subset"]

trees, pdata = mu.setupPionData(inputpath, rootfiles)

In [6]:
np0 = len(pdata['pi0_subset'])
nppm = len(pdata['pi_charged_subset'])

print("Number of pi0 events: {}".format(np0))
print("Number of pi+/- events: {}".format(nppm))
print("Total: {}".format(np0+nppm))

Number of pi0 events: 1243504
Number of pi+/- events: 1261440
Total: 2504944


In [7]:
pcells = {
    ifile : {
        layer : mu.setupCells(itree, layer, flatten = False)
        for layer in mu.cell_meta
    }
    for ifile, itree in trees.items()
}

## Can we train the CNN still?

In [8]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow import keras as keras

In [9]:
import tensorflow 
NUM_THREADS=1
tensorflow.config.threading.set_inter_op_parallelism_threads(NUM_THREADS)
tensorflow.config.threading.set_intra_op_parallelism_threads(NUM_THREADS)

gpu_list = ["/gpu:0"]
#strategy = tf.distribute.MirroredStrategy(devices=gpu_list)
# strategy = tf.distribute.MirroredStrategy()
#ngpu = strategy.num_replicas_in_sync
#print ('Number of devices: {}'.format(ngpu))

In [10]:
training_classes = ['pi0_subset','pi_charged_subset']
# create train/validation/test subsets containing 70%/10%/20%
# of events from each type of pion event
pdata_merged, pcells_merged, plabels = mu.createTrainingDatasets(training_classes, pdata, pcells)

In [11]:
pcells_merged_reshaped = mu.reshapeSeparateCNN(pcells_merged)

In [12]:
# not sure how useful these 1,1 filters are?
# may need 'shallower' networks instead for the further networks
filters = {
    'EMB1': (2,4), 
    'EMB2': (4,4), 
    'EMB3': (4,2), 
    'TileBar0': (2,2), 
    'TileBar1': (2,2), 
    'TileBar2': (1,1)
}
pools2 = {
    'EMB1': (2,2), 
    'EMB2': (2,2), 
    'EMB3': (1,1), 
    'TileBar0': (1,1), 
    'TileBar1': (1,1), 
    'TileBar2': (1,1)
}

In [13]:
def cnn_model_layers(layer):
    print(layer)
    # create model
    #with strategy.scope():
    model = Sequential()
    #model.add(Convolution2D(32, filters[layer], input_shape=(1,mu.cell_meta[layer]['len_eta'],mu.cell_meta[layer]['len_phi']), activation='relu', data_format = 'channels_last'))
    model.add(Convolution2D(32, filters[layer], input_shape=(mu.cell_meta[layer]['len_eta'],mu.cell_meta[layer]['len_phi'],1),activation='relu', data_format = 'channels_first'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    # model.add(Convolution2D(16, (2, 2), activation='relu'))
    model.add(Convolution2D(16, pools2[layer], activation='relu'))
    model.add(MaxPool2D(pool_size=pools2[layer]))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(2, kernel_initializer='normal', activation='softmax'))
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [14]:
models = {
    layer: cnn_model_layers(layer)
    for layer in mu.cell_meta
}

EMB1


ValueError: Negative dimension size caused by subtracting 4 from 1 for '{{node conv2d/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d/Conv2D/ReadVariableOp)' with input shapes: [?,128,4,1], [2,4,128,32].

In [ ]:
history = {
    layer: models[layer].fit(pcells_merged_reshaped[layer][pdata_merged.train], 
                                plabels[pdata_merged.train], 
                                validation_data=(pcells_merged_reshaped[layer][pdata_merged.val], plabels[pdata_merged.val]), epochs=100, batch_size=200, verbose=2)
    for layer in mu.cell_meta
}

In [ ]:
performance = {}
scores = {}
for layer in mu.cell_meta:
    print('On layer ' + layer)
    
    # get overall performance metric
    performance[layer] = models[layer].evaluate(
        pcells_merged_reshaped[layer][pdata_merged.test], plabels[pdata_merged.test],
        verbose = 0,
    )
    
    # get network scores for the dataset
    scores[layer] = models[layer].predict(
        pcells_merged_reshaped[layer]
    )
    
    print('Finished layer ' + layer)


In [ ]:
import pickle

for layer, model in models.items():
    model.save(modelpath+"model_cnn_"+layer+".h5")
    with open(modelpath + "model_cnn_"+layer+".history",'wb') as model_history_file:
        pickle.dump(history[layer].history, model_history_file)

In [ ]:
import pickle

models = {}
history = {}
for layer in mu.cell_meta:
    models[layer] = tf.keras.models.load_model(modelpath+"model_cnn_"+layer+".h5")
    with open(modelpath + 'model_cnn_'+layer+'.history','rb') as model_history_file:
        history[laye] = pickle.load(model_history_file)

In [ ]:
for layer in mu.cell_meta:
#     print(history_flat[layer_i].history.keys())
    plt.cla(); plt.clf()
    fig = plt.figure()
    fig.patch.set_facecolor('white')

    plt.plot(history[layer].history['acc'])
    plt.plot(history[layer].history['val_acc'])
    plt.title('model accuracy for ' + layer)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig('Plots/accuracy_' + layer + '.pdf')
    plt.show()


    # summarize history for loss
    fig = plt.figure()
    fig.patch.set_facecolor('white')
    plt.plot(history[layer].history['loss'])
    plt.plot(history[layer].history['val_loss'])
    plt.title('model loss for ' + layer)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig(plotpath + 'loss_' + layer + '.pdf')
    plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

roc_fpr = {}
roc_tpr = {}
roc_thresh = {}
roc_auc = {}

for layer in mu.cell_meta:
    roc_fpr[layer], roc_tpr[layer], roc_thresh[layer] = roc_curve(
        plabels[pdata_merged.test][:,1],
        scores[layer][pdata_merged.test,1],
        drop_intermediate=False,
    )
    roc_auc[layer] = auc(roc_fpr[layer], roc_tpr[layer])
    print('Area under curve for ' + layer + ': ' + str(roc_auc[layer]))

In [ ]:
pu.roc_plot([roc_fpr[layer] for layer in mu.cell_meta],
            [roc_tpr[layer] for layer in mu.cell_meta],
            figfile=plotpath + 'roc_cnn_all.pdf',
            labels=['{} (area = {:.3f})'.format(layer, roc_auc[layer]) for layer in mu.cell_meta],
            extra_lines=[[[0, 1], [0, 1]]],
            title='CNN ROC curve: classification of $\pi^+$ vs. $\pi^0$')

## More with CNN's

In [ ]:
def merged_model_emb123():
    with strategy.scope():
        # EMB1 image (flat, fully-connected)
        input1 = Input(shape=(1, 128, 4), name='input1')
        x1 = Convolution2D(32, (4, 2), activation='relu', data_format = 'channels_first')(input1)
        x1 = MaxPool2D(pool_size=(2, 2))(x1)
        x1 = Dropout(0.2)(x1)
        x1 = Flatten()(x1)
        x1 = Dense(128, activation='relu')(x1)

        # EMB2 image (convolutional)
        input2 = Input(shape=(1,16,16), name='input2')
        x2 = Convolution2D(32, (4, 4), activation='relu', data_format = 'channels_first')(input2)
        x2 = MaxPool2D(pool_size=(2, 2))(x2)
        x2 = Dropout(0.2)(x2)
        x2 = Flatten()(x2)
        x2 = Dense(128, activation='relu')(x2)
    
        # EMB3 image (convolutional)
        input3 = Input(shape=(1,8,16), name='input3')
        x3 = Convolution2D(32, (2, 4), activation='relu', data_format = 'channels_first')(input3)
        x3 = MaxPool2D(pool_size=(1, 2))(x3)
        x3 = Dropout(0.2)(x3)
        x3 = Flatten()(x3)
        x3 = Dense(128, activation='relu')(x3)

        # concatenate outputs from the three networks above
        x = concatenate([x1, x2, x3]) 
        x = Dense(50, activation='relu')(x)    

        # final output
        output = Dense(2, activation='softmax')(x)

        model = Model(inputs = [input1, input2, input3], outputs = [output])
    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
        return model

In [ ]:
merged_emb123 = merged_model_emb123()


In [ ]:
tf.keras.utils.plot_model(merged_emb123, plotpath+'emb123_with_shape_info.png', show_shapes=True)


In [ ]:
history_emb123 = merged_emb123.fit([pcells_merged_reshaped['EMB1'][pdata_merged.train],
                                    pcells_merged_reshaped['EMB2'][pdata_merged.train],
                                    pcells_merged_reshaped['EMB3'][pdata_merged.train]],
                                    plabels[pdata_merged.train], 
                                    validation_data=([pcells_merged_reshaped['EMB1'][pdata_merged.val],
                                                        pcells_merged_reshaped['EMB2'][pdata_merged.val],
                                                        pcells_merged_reshaped['EMB3'][pdata_merged.val]], 
                                                        plabels[pdata_merged.val]),                            
                                    epochs=250, batch_size=200*ngpu, verbose=2)

In [ ]:
merged_emb123.save(modelpath+'model_emb123.h5')

In [ ]:
    plt.cla(); plt.clf()
    fig = plt.figure()
    fig.patch.set_facecolor('white')

    plt.plot(history_emb123.history['accuracy'])
    plt.plot(history_emb123.history['val_accuracy'])
    plt.title('model accuracy for EMB123')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig('Plots/accuracy_' + layer + '.pdf')
    plt.show()


    # summarize history for loss
    fig = plt.figure()
    fig.patch.set_facecolor('white')
    plt.plot(history_emb123.history['loss'])
    plt.plot(history_emb123.history['val_loss'])
    plt.title('model loss for emb123')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig(plotpath + 'loss_' + layer + '.pdf')
    plt.show()

In [ ]:
scores_123 = merged_emb123.predict(
        [pcells_merged_reshaped['EMB1'], pcells_merged_reshaped['EMB2'], pcells_merged_reshaped['EMB3']]
        )

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_fpr_123, roc_tpr_123, roc_thresh_123 = roc_curve(
        plabels[pdata_merged.test][:,1],
        scores_123[pdata_merged.test,1],
        drop_intermediate=False,
    )
roc_auc_123 = auc(roc_fpr_123, roc_tpr_123)
print('Area under curve for EMB123 ' + str(roc_auc_123))

In [ ]:
plt.cla(); plt.clf()
# fig, ax = plt.subplots(1, 2, tight_layout=True, figsize=(10,4))
# fig.patch.set_facecolor('white')

fig = plt.figure()
fig.patch.set_facecolor('white')
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(roc_fpr_123, roc_tpr_123, label='CNN (area = {:.3f})'.format(roc_auc_123))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('CNN ROC curve: classification of $\pi^+$ vs. $\pi^0$')
plt.legend(loc='best')
# plt.savefig('Plots/roc_lc_only.pdf')
plt.show()

In [ ]:
def merged_model_all():
    with strategy.scope():
        # EMB1 image (convolutional)
        input1 = Input(shape=(1, 128, 4), name='input1')
        x1 = Convolution2D(32, (4, 2), activation='relu', data_format = 'channels_first')(input1)
        x1 = MaxPool2D(pool_size=(2, 2))(x1)
        x1 = Dropout(0.2)(x1)
        x1 = Flatten()(x1)
        x1 = Dense(128, activation='relu')(x1)

        # EMB2 image (convolutional)
        input2 = Input(shape=(1,16,16), name='input2')
        x2 = Convolution2D(32, (4, 4), activation='relu', data_format = 'channels_first')(input2)
        x2 = MaxPool2D(pool_size=(2, 2))(x2)
        x2 = Dropout(0.2)(x2)
        x2 = Flatten()(x2)
        x2 = Dense(128, activation='relu')(x2)
    
        # EMB3 image (convolutional)
        input3 = Input(shape=(1,8,16), name='input3')
        x3 = Convolution2D(32, (2, 4), activation='relu', data_format = 'channels_first')(input3)
        x3 = MaxPool2D(pool_size=(2, 2))(x3)
        x3 = Dropout(0.2)(x3)
        x3 = Flatten()(x3)
        x3 = Dense(128, activation='relu')(x3)

        input4 = Input(shape=(1,4,4), name='input4')
        x4 = Convolution2D(32, (2,2), activation='relu', data_format = 'channels_first')(input4)
        x4 = MaxPool2D(pool_size=(2,2))(x4)
        x4 = Dropout(0.2)(x4)
        x4 = Flatten()(x4)
        x4 = Dense(128, activation='relu')(x4)

        input5 = Input(shape=(1,4,4), name='input5')
        x5 = Convolution2D(32, (2,2), activation='relu', data_format = 'channels_first')(input5)
        x5 = MaxPool2D(pool_size=(2,2))(x5)
        x5 = Dropout(0.2)(x5)
        x5 = Flatten()(x5)
        x5 = Dense(128, activation='relu')(x5)

        input6 = Input(shape=(1,2,4), name='input6')
        x6 = Convolution2D(32, (2,2), activation='relu', data_format = 'channels_first')(input6)
        x6 = MaxPool2D(pool_size=(2,1))(x6)
        x6 = Dropout(0.2)(x6)
        x6 = Flatten()(x6)
        x6 = Dense(128, activation='relu')(x6)

        # concatenate outputs from the three networks above
        x = concatenate([x1, x2, x3, x4, x5, x6]) 
        x = Dense(50, activation='relu')(x)    

        # final output
        output = Dense(2, activation='softmax')(x)

        model = Model(inputs = [input1, input2, input3, input4, input5, input6], outputs = [output])
    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
        return model

In [ ]:
merged_all = merged_model_all()
merged_all.summary()


In [ ]:
tf.keras.utils.plot_model(merged_all, plotpath+'all_with_shape_info.png', show_shapes=True)

In [ ]:
history_all = merged_all.fit([pcells_merged_reshaped['EMB1'][pdata_merged.train],
                                    pcells_merged_reshaped['EMB2'][pdata_merged.train],
                                    pcells_merged_reshaped['EMB3'][pdata_merged.train],
                                    pcells_merged_reshaped['TileBar0'][pdata_merged.train],
                                    pcells_merged_reshaped['TileBar1'][pdata_merged.train],
                                    pcells_merged_reshaped['TileBar2'][pdata_merged.train]],
                                    plabels[pdata_merged.train], 
                                    validation_data=([pcells_merged_reshaped['EMB1'][pdata_merged.val],
                                            pcells_merged_reshaped['EMB2'][pdata_merged.val],
                                            pcells_merged_reshaped['EMB3'][pdata_merged.val],
                                            pcells_merged_reshaped['TileBar0'][pdata_merged.val],
                                            pcells_merged_reshaped['TileBar1'][pdata_merged.val],
                                            pcells_merged_reshaped['TileBar2'][pdata_merged.val]], 
                                            plabels[pdata_merged.val]),                            
                                    epochs=250, batch_size=200*ngpu, verbose=2)
merged_all.save(modelpath+'model_cnn_all.h5')
scores_all = merged_all.predict(
        [pcells_merged_reshaped['EMB1'], pcells_merged_reshaped['EMB2'], pcells_merged_reshaped['EMB3'],pcells_merged_reshaped['TileBar0'], pcells_merged_reshaped['TileBar1'], pcells_merged_reshaped['TileBar2']]
        )

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_fpr_all, roc_tpr_all, roc_thresh_all = roc_curve(
        plabels[pdata_merged.test][:,1],
        scores_all[pdata_merged.test,1],
        drop_intermediate=False,
    )
roc_auc_all = auc(roc_fpr_all, roc_tpr_all)
print('Area under curve for CNN All ' + str(roc_auc_all))

In [ ]:
plt.cla(); plt.clf()
# fig, ax = plt.subplots(1, 2, tight_layout=True, figsize=(10,4))
# fig.patch.set_facecolor('white')

fig = plt.figure()
fig.patch.set_facecolor('white')
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(roc_fpr_123, roc_tpr_123, label='CNN EMB (area = {:.3f})'.format(roc_auc_123))
plt.plot(roc_fpr_all, roc_tpr_all, label='CNN all (area = {:.3f})'.format(roc_auc_all))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('CNN ROC curve: classification of $\pi^+$ vs. $\pi^0$')
plt.legend(loc='best')
# plt.savefig('Plots/roc_lc_only.pdf')
plt.show()

## Ok, now we play with more complicated models...

In [ ]:
import scipy.ndimage

In [ ]:
iterEMB2 = (zoom(cluster) for cluster in pcells_merged['EMB2'])

In [ ]:
next(iterEMB2)

In [ ]:
upscaledEMB3 = scipy.ndimage.zoom(pcells_merged['EMB3'], (1,2,1))

In [ ]:
upscaledEMB3.shape

In [ ]:
downScaledEMB1 = scipy.ndimage.zoom(pcells_merged['EMB1'], (1,0.125,4))

In [ ]:
downScaledEMB1.shape

In [ ]:
test = (1,2)

In [ ]:
pcells_EMB2G_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (16, 16)))

In [ ]:
pcells_EMB2G_channels.shape

In [ ]:
def channels_EMB2G_model_all():
    with strategy.scope():
        # Here come the 16x16 inputs
        input1 = Input(shape=(6, 16, 16), name='input1')
        x1 = Convolution2D(32, (4, 4), activation='relu', data_format = 'channels_first')(input1)
        x1 = MaxPool2D(pool_size=(2, 2))(x1)
        x1 = Dropout(0.2)(x1)
        x1 = Flatten()(x1)
        x1 = Dense(128, activation='relu')(x1)


        # final output
        output = Dense(2, activation='softmax')(x1)

        model = Model(inputs = [input1], outputs = [output])
    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
        return model

In [ ]:
model_channels = channels_EMB2G_model_all()
model_channels.summary()

In [ ]:
history_channels = model_channels.fit([pcells_EMB2G_channels[pdata_merged.train]],
                                    plabels[pdata_merged.train], 
                                    validation_data=([pcells_EMB2G_channels[pdata_merged.val]], 
                                            plabels[pdata_merged.val]),                            
                                    epochs=250, batch_size=200*ngpu, verbose=2)

In [ ]:
print('test')

In [ ]:
model_channels.save(modelpath+'model_cnn_channels.h5')


In [ ]:
scores_chan = model_channels.predict(
        [pcells_EMB2G_channels]
        )

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_fpr_chan, roc_tpr_chan, roc_thresh_chan = roc_curve(
        plabels[pdata_merged.test][:,1],
        scores_chan[pdata_merged.test,1],
        drop_intermediate=False,
    )
roc_auc_chan = auc(roc_fpr_chan, roc_tpr_chan)
print('Area under curve for CNN All ' + str(roc_auc_chan))

0.981 for no drop-out, but lots of overfitting

In [ ]:
    plt.cla(); plt.clf()
    fig = plt.figure()
    fig.patch.set_facecolor('white')

    plt.plot(history_channels.history['accuracy'])
    plt.plot(history_channels.history['val_accuracy'])
    plt.title('model accuracy for Channels')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig('Plots/accuracy_' + layer + '.pdf')
    plt.show()


    # summarize history for loss
    fig = plt.figure()
    fig.patch.set_facecolor('white')
    plt.plot(history_channels.history['loss'])
    plt.plot(history_channels.history['val_loss'])
    plt.title('model loss for Channels')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig(plotpath + 'loss_' + layer + '.pdf')
    plt.show()

### Let's test the largest granularity possible

In [ ]:
pcells_GG_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (128, 16)))

In [ ]:
def channels_GG_model_all():
    with strategy.scope():
        # Here come the 16x16 inputs
        input1 = Input(shape=(6, 128, 16), name='input1')
        x1 = Convolution2D(32, (4, 4), activation='relu', data_format = 'channels_first')(input1)
        x1 = MaxPool2D(pool_size=(2, 2))(x1)
        x1 = Dropout(0.2)(x1)
        x1 = Flatten()(x1)
        x1 = Dense(128, activation='relu')(x1)


        # final output
        output = Dense(2, activation='softmax')(x1)

        model = Model(inputs = [input1], outputs = [output])
    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
        return model

### Ok that doesn't work because it's too big :D or at least I'm not patient enough

Instead let's try something simpler...

In [ ]:
pcells_EMB2_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (16, 16)))

In [ ]:
pcells_merged['EMB1'].shape

In [ ]:
pcells_EMB1_flat = pcells_merged['EMB1'].reshape(len(pcells_merged['EMB1']), 128 * 4)

In [ ]:
def channels_EMB2G_EMB1F_model_all():
    with strategy.scope():
        # Here come the 16x16 inputs
        input1 = Input(shape=(6, 16, 16), name='input1')
        x1 = Convolution2D(32, (4, 4), activation='relu', data_format = 'channels_first')(input1)
        x1 = MaxPool2D(pool_size=(2, 2))(x1)
        x1 = Dropout(0.2)(x1)
        x1 = Flatten()(x1)
        x1 = Dense(128, activation='relu')(x1)

        emb1_dim = 128*4
        #Here's EMB1 flattened
        input2 = Input(shape=(emb1_dim), name='input2')
        x2 = Dense(emb1_dim, activation='relu')(input2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 2, activation='relu')(x2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 4, activation='relu')(x2)

        # concatenate outputs from the two networks above
        x = concatenate([x1, x2]) 
        x = Dense(50, activation='relu')(x)    

        # final output
        output = Dense(2, activation='softmax')(x)

        model = Model(inputs = [input1,input2], outputs = [output])
    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
        return model

In [ ]:
model_EMB2G_EMB1F = channels_EMB2G_EMB1F_model_all()
model_EMB2G_EMB1F.summary()

In [ ]:
history_EMB2G_EMB1F = model_EMB2G_EMB1F.fit([pcells_EMB2_channels[pdata_merged.train], pcells_EMB1_flat[pdata_merged.train]],
                                    plabels[pdata_merged.train], 
                                    validation_data=([pcells_EMB2_channels[pdata_merged.val], pcells_EMB1_flat[pdata_merged.val]], 
                                            plabels[pdata_merged.val]),                            
                                    epochs=250, batch_size=200*ngpu, verbose=2)
model_EMB2G_EMB1F.save(modelpath+'model_EMB2G_EMB1F.h5')
scores_EMB2G_EMB1F = model_EMB2G_EMB1F.predict([pcells_EMB2_channels, pcells_EMB1_flat])

In [ ]:
    plt.cla(); plt.clf()
    fig = plt.figure()
    fig.patch.set_facecolor('white')

    plt.plot(history_EMB2G_EMB1F.history['accuracy'])
    plt.plot(history_EMB2G_EMB1F.history['val_accuracy'])
    plt.title('model accuracy for Channels')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig('Plots/accuracy_' + layer + '.pdf')
    plt.show()


    # summarize history for loss
    fig = plt.figure()
    fig.patch.set_facecolor('white')
    plt.plot(history_EMB2G_EMB1F.history['loss'])
    plt.plot(history_EMB2G_EMB1F.history['val_loss'])
    plt.title('model loss for Channels')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig(plotpath + 'loss_' + layer + '.pdf')
    plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_fpr_EMB2G_EMB1F, roc_tpr_EMB2G_EMB1F, roc_thresh_EMB2G_EMB1F = roc_curve(
        plabels[pdata_merged.test][:,1],
        scores_EMB2G_EMB1F[pdata_merged.test,1],
        drop_intermediate=False,
    )
roc_auc_EMB2G_EMB1F = auc(roc_fpr_EMB2G_EMB1F, roc_tpr_EMB2G_EMB1F)
print('Area under curve for CNN EMB2G_EMB1F ' + str(roc_auc_EMB2G_EMB1F))

In [ ]:
plt.cla(); plt.clf()
# fig, ax = plt.subplots(1, 2, tight_layout=True, figsize=(10,4))
# fig.patch.set_facecolor('white')

fig = plt.figure()
fig.patch.set_facecolor('white')
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(roc_fpr_EMB2G_EMB1F, roc_tpr_EMB2G_EMB1F, label='CNN EMB2G EMB1F (area = {:.3f})'.format(roc_auc_EMB2G_EMB1F))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('CNN ROC curve: classification of $\pi^+$ vs. $\pi^0$')
plt.legend(loc='best')
plt.savefig(plotpath+'/roc_EMB2G_EMB1F_only.pdf')
plt.show()

In [ ]:
def channels_EMB2G_EMB1F_model_all2():
    with strategy.scope():
        # Here come the 16x16 inputs
        input1 = Input(shape=(6, 16, 16), name='input1')
        x1 = Convolution2D(32, (4, 4), activation='relu', data_format = 'channels_first')(input1)
        # x1 = MaxPool2D(pool_size=(2, 2))(x1)
        x1 = Flatten()(x1)
        x1 = Dense(128, activation='relu')(x1)
        x1 = Dropout(0.2)(x1)
        x1 = Dense(64, activation='relu')(x1)

        emb1_dim = 128*4
        #Here's EMB1 flattened
        input2 = Input(shape=(emb1_dim), name='input2')
        x2 = Dense(emb1_dim, activation='relu')(input2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 2, activation='relu')(x2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 4, activation='relu')(x2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 8, activation='relu')(x2)

        # concatenate outputs from the two networks above
        x = concatenate([x1, x2]) 
        x = Dense(50, activation='relu')(x)    

        # final output
        output = Dense(2, activation='softmax')(x)

        model = Model(inputs = [input1,input2], outputs = [output])
    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
        return model

In [ ]:
model_EMB2G_EMB1F_2 = channels_EMB2G_EMB1F_model_all2()
model_EMB2G_EMB1F_2.summary()

In [ ]:
history_EMB2G_EMB1F_2 = model_EMB2G_EMB1F_2.fit([pcells_EMB2_channels[pdata_merged.train], pcells_EMB1_flat[pdata_merged.train]],
                                    plabels[pdata_merged.train], 
                                    validation_data=([pcells_EMB2_channels[pdata_merged.val], pcells_EMB1_flat[pdata_merged.val]], 
                                            plabels[pdata_merged.val]),                            
                                    epochs=250, batch_size=200*ngpu, verbose=2)
model_EMB2G_EMB1F_2.save(modelpath+'model_EMB2G_EMB1F_2.h5')
scores_EMB2G_EMB1F_2 = model_EMB2G_EMB1F_2.predict([pcells_EMB2_channels, pcells_EMB1_flat])

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_fpr_EMB2G_EMB1F_2, roc_tpr_EMB2G_EMB1F_2, roc_thresh_EMB2G_EMB1F_2 = roc_curve(
        plabels[pdata_merged.test][:,1],
        scores_EMB2G_EMB1F_2[pdata_merged.test,1],
        drop_intermediate=False,
    )
roc_auc_EMB2G_EMB1F_2 = auc(roc_fpr_EMB2G_EMB1F_2, roc_tpr_EMB2G_EMB1F_2)
print('Area under curve for CNN EMB2G_EMB1F 2 ' + str(roc_auc_EMB2G_EMB1F_2))

More fun!

In [ ]:
def channels_EMB2G_EMB1F_model_all3():
    with strategy.scope():
        # Here come the 16x16 inputs
        input1 = Input(shape=(6, 16, 16), name='input1')
        x1 = Convolution2D(32, (4, 4), activation='relu', data_format = 'channels_first')(input1)
        # x1 = MaxPool2D(pool_size=(2, 2))(x1)
        x1 = Flatten()(x1)
        x1 = Dense(128, activation='relu')(x1)
        x1 = Dropout(0.2)(x1)
        x1 = Dense(64, activation='relu')(x1)

        # this layer corresponds to physical granularity of EMB2
        x1_EMB2 = MaxPool2D(pool_size=(2, 1))(input1)
        x1_EMB2 = Convolution2D(32, (2, 2), activation='relu', data_format = 'channels_first')(x1_EMB2)
        x1_EMB2 = Flatten()(x1_EMB2)
        x1_EMB2 = Dense(128, activation='relu')(x1_EMB2)
        x1_EMB2 = Dropout(0.2)(x1_EMB2)
        x1_EMB2 = Dense(64, activation='relu')(x1_EMB2)

        # this layer corresponds to physical granularity of TileBar0
        x1_T0 = MaxPool2D(pool_size=(2, 2))(input1)
        x1_T0 = Convolution2D(32, (2, 2), activation='relu', data_format = 'channels_first')(x1_T0)
        x1_T0 = Flatten()(x1_T0)
        x1_T0 = Dense(128, activation='relu')(x1_T0)
        x1_T0 = Dropout(0.2)(x1_T0)
        x1_T0 = Dense(64, activation='relu')(x1_T0)

        emb1_dim = 128*4
        #Here's EMB1 flattened
        input2 = Input(shape=(emb1_dim), name='input2')
        x2 = Dense(emb1_dim, activation='relu')(input2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 2, activation='relu')(x2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 4, activation='relu')(x2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 8, activation='relu')(x2)

        # concatenate outputs from the two networks above
        x = concatenate([x1,x1_EMB2,x1_T0, x2]) 
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)    
        x = Dense(64, activation='relu')(x)

        # final output
        output = Dense(2, activation='softmax')(x)

        model = Model(inputs = [input1,input2], outputs = [output])
    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
        return model

In [ ]:
model_EMB2G_EMB1F_3 = channels_EMB2G_EMB1F_model_all3()
model_EMB2G_EMB1F_3.summary()

In [ ]:
history_EMB2G_EMB1F_3 = model_EMB2G_EMB1F_3.fit([pcells_EMB2_channels[pdata_merged.train], pcells_EMB1_flat[pdata_merged.train]],
                                    plabels[pdata_merged.train], 
                                    validation_data=([pcells_EMB2_channels[pdata_merged.val], pcells_EMB1_flat[pdata_merged.val]], 
                                            plabels[pdata_merged.val]),                            
                                    epochs=250, batch_size=200*ngpu, verbose=2)
model_EMB2G_EMB1F_3.save(modelpath+'model_EMB2G_EMB1F_3.h5')
scores_EMB2G_EMB1F_3 = model_EMB2G_EMB1F_3.predict([pcells_EMB2_channels, pcells_EMB1_flat])

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_fpr_EMB2G_EMB1F_3, roc_tpr_EMB2G_EMB1F_3, roc_thresh_EMB2G_EMB1F_3 = roc_curve(
        plabels[pdata_merged.test][:,1],
        scores_EMB2G_EMB1F_3[pdata_merged.test,1],
        drop_intermediate=False,
    )
roc_auc_EMB2G_EMB1F_3 = auc(roc_fpr_EMB2G_EMB1F_3, roc_tpr_EMB2G_EMB1F_3)
print('Area under curve for CNN EMB2G_EMB1F 3 ' + str(roc_auc_EMB2G_EMB1F_3))

In [ ]:
    plt.cla(); plt.clf()
    fig = plt.figure()
    fig.patch.set_facecolor('white')

    plt.plot(history_EMB2G_EMB1F_3.history['accuracy'])
    plt.plot(history_EMB2G_EMB1F_3.history['val_accuracy'])
    plt.title('model accuracy for Channels')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig('Plots/accuracy_' + layer + '.pdf')
    plt.show()


    # summarize history for loss
    fig = plt.figure()
    fig.patch.set_facecolor('white')
    plt.plot(history_EMB2G_EMB1F_3.history['loss'])
    plt.plot(history_EMB2G_EMB1F_3.history['val_loss'])
    plt.title('model loss for Channels')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig(plotpath + 'loss_' + layer + '.pdf')
    plt.show()

In [ ]:
def channels_EMB2G_EMB1F_model_all4():
    with strategy.scope():
        # Here come the 16x16 inputs
        input1 = Input(shape=(6, 16, 16), name='input1')

        x1 = Convolution2D(32, (4, 4), activation='relu', data_format = 'channels_first')(input1)
        x1 = MaxPool2D(pool_size=(2, 2))(x1)
        x1 = Convolution2D(32, (2, 2), activation='relu')(x1)
        x1 = MaxPool2D(pool_size=(2, 2))(x1)
        x1 = Flatten()(x1)
        x1 = Dense(128, activation='relu')(x1)
        x1 = Dropout(0.2)(x1)
        x1 = Dense(64, activation='relu')(x1)

        x1_2 = Convolution2D(64, (2, 2), activation='relu', data_format = 'channels_first')(input1)
        x1_2 = MaxPool2D(pool_size=(2, 2))(x1_2)
        x1_2 = Convolution2D(32, (2, 2), activation='relu')(x1_2)
        x1_2 = MaxPool2D(pool_size=(2, 2))(x1_2)
        x1_2 = Flatten()(x1_2)
        x1_2 = Dense(128, activation='relu')(x1_2)
        x1_2 = Dropout(0.2)(x1_2)
        x1_2 = Dense(64, activation='relu')(x1_2)

        emb1_dim = 128*4
        #Here's EMB1 flattened
        input2 = Input(shape=(emb1_dim), name='input2')
        x2 = Dense(emb1_dim, activation='relu')(input2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 2, activation='relu')(x2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 4, activation='relu')(x2)
        x2 = Dropout(0.2)(x2)
        x2 = Dense(emb1_dim / 8, activation='relu')(x2)

        # concatenate outputs from the two networks above
        x = concatenate([x1, x1_2, x2]) 
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)    
        x = Dense(64, activation='relu')(x)  

        # final output
        output = Dense(2, activation='softmax')(x)

        model = Model(inputs = [input1,input2], outputs = [output])
    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
        return model

In [ ]:
model_EMB2G_EMB1F_4 = channels_EMB2G_EMB1F_model_all4()
model_EMB2G_EMB1F_4.summary()

In [ ]:
history_EMB2G_EMB1F_4 = model_EMB2G_EMB1F_4.fit([pcells_EMB2_channels[pdata_merged.train], pcells_EMB1_flat[pdata_merged.train]],
                                    plabels[pdata_merged.train], 
                                    validation_data=([pcells_EMB2_channels[pdata_merged.val], pcells_EMB1_flat[pdata_merged.val]], 
                                            plabels[pdata_merged.val]),                            
                                    epochs=250, batch_size=200*ngpu, verbose=2)
model_EMB2G_EMB1F_4.save(modelpath+'model_EMB2G_EMB1F_4.h5')
scores_EMB2G_EMB1F_4 = model_EMB2G_EMB1F_4.predict([pcells_EMB2_channels, pcells_EMB1_flat])

Ok, from the accuracy it's clear this isn't beating 2, which is our best network. Ok, let's try RESNET?

## RESNET TIME

In [ ]:
# copied from https://keras.io/examples/cifar10_resnet/
# modified to use tf.keras layer names
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Convolution2D(num_filters,
                    kernel_size=kernel_size,
                    strides=strides,
                    padding='same',
                    kernel_initializer='he_normal',
                    kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, final_pool = 8, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=final_pool)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, final_pool, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=final_pool)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
pcells_EMB2_channels_last = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (16, 16)), last=True)

In [ ]:
input_shape = pcells_EMB2_channels_last.shape[1:]
batch_size = 128  # orig paper trained all networks with batch_size=128
epochs = 100
num_classes = 2

In [ ]:
version = 2
n = 5
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2
model_type = 'ResNet%dv%d' % (depth, version)

if version == 2:
    model_ResNet47v2 = resnet_v2(input_shape=input_shape, final_pool=4, depth=depth, num_classes=num_classes)
else:
    model_ResNet47v2 = resnet_v1(input_shape=input_shape, final_pool=4, depth=depth, num_classes=num_classes)

model_ResNet47v2.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model_ResNet47v2.summary()
print(model_type)

In [ ]:
history_ResNet47v2 = model_ResNet47v2.fit([pcells_EMB2_channels_last[pdata_merged.train]],
                                    plabels[pdata_merged.train], 
                                    validation_data=([pcells_EMB2_channels_last[pdata_merged.val]], 
                                            plabels[pdata_merged.val]),                            
                                    epochs=epochs, batch_size=batch_size, verbose=2)
model_ResNet47v2.save(modelpath+'model_ResNet47v2.h5')


In [ ]:
model_ResNet47v2 = tf.keras.models.load_model(modelpath+'model_ResNet47v2.h5')

In [ ]:
scores_ResNet20v1 = model.predict([pcells_EMB2_channels_last])

In [ ]:
scores_ResNet47v2 = model_ResNet47v2.predict([pcells_EMB2_channels_last])

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_fpr_ResNet20v1, roc_tpr_ResNet20v1, roc_thresh_ResNet20v1 = roc_curve(
        plabels[pdata_merged.test][:,1],
        scores_ResNet20v1[pdata_merged.test,1],
        drop_intermediate=False,
    )
roc_auc_ResNet20v1 = auc(roc_fpr_ResNet20v1, roc_tpr_ResNet20v1)
print('Area under curve for ResNet20v1 ' + str(roc_auc_ResNet20v1))

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_fpr_ResNet47v2, roc_tpr_ResNet47v2, roc_thresh_ResNet47v2 = roc_curve(
        plabels[pdata_merged.train][:,1],
        scores_ResNet47v2[pdata_merged.train,1],
        drop_intermediate=False,
    )
roc_auc_ResNet47v2 = auc(roc_fpr_ResNet47v2, roc_tpr_ResNet47v2)
print('Area under curve for ResNet47v2 ' + str(roc_auc_ResNet47v2))

In [ ]:
    plt.cla(); plt.clf()
    fig = plt.figure()
    fig.patch.set_facecolor('white')

    plt.plot(history_ResNet20v1.history['accuracy'])
    plt.plot(history_ResNet20v1.history['val_accuracy'])
    plt.title('model accuracy for Channels')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig('Plots/accuracy_' + layer + '.pdf')
    plt.show()


    # summarize history for loss
    fig = plt.figure()
    fig.patch.set_facecolor('white')
    plt.plot(history_ResNet20v1.history['loss'])
    plt.plot(history_ResNet20v1.history['val_loss'])
    plt.title('model loss for Channels')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # plt.savefig(plotpath + 'loss_' + layer + '.pdf')
    plt.show()

In [ ]:
plt.cla(); plt.clf()
# fig, ax = plt.subplots(1, 2, tight_layout=True, figsize=(10,4))
# fig.patch.set_facecolor('white')

fig = plt.figure()
fig.patch.set_facecolor('white')
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(roc_fpr_ResNet20v1, roc_tpr_ResNet20v1, label='ResNet20v1 (area = {:.3f})'.format(roc_auc_ResNet20v1))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ResNet20v1 ROC curve: classification of $\pi^+$ vs. $\pi^0$')
plt.legend(loc='best')
plt.savefig(plotpath+'/roc_ResNet20v1.pdf')
plt.show()